<a href="https://colab.research.google.com/github/Fredf23/TatChim/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22kaggle_compound_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Данный ноутбук исследует возможность применения датасета, доступного на Kaggle.
## Указанный датасет (https://www.kaggle.com/datasets/cnezhmar/omposite-materials) содержит исходные признаки, выраженные в виде пропорций матрицы-наполнителя, количества использованного отвердителя и др., а также физико-химические показатели произведенного компаунда: плотность, модуль упругости, прочности и тд.
## Поэтому было принято решение разделить датасет на входные и целевые показатели:
## Входные признаки:
###   *-Соотношение матрица-наполнитель*
### *-Плотность, кг/м3*
###     *-Количество отвердителя, м.%*
### *-Содержание эпоксидных групп,%_2*
### *-Потребление смолы, г/м2*
###
## К целевым признакам были отнесены:
### *-Модуль упругости, ГПа*
### *-Температура вспышки, С_2*
### *-Поверхностная плотность, г/м2*
###  *-Модуль упругости при растяжении, ГПа*
### *-Прочность при растяжении, МПа*



# 1. Парсинг датасета, проверка на пропуски,


### Так как оригинальный датасет представлен в формате xlsx, после скачивания файл был конвертирован/сохранен в формат csv. При этом был указан разделитель "," (comma).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
file_path = 'укажите ваш путь к скачанному в формате csv файлу'
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
0,0,1.857143,2030.0,738.736842,30.00,22.267857,100.000000,210.0,70.0,3000.0,220.0
1,1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0
2,2,1.857143,2030.0,738.736842,49.90,33.000000,284.615385,210.0,70.0,3000.0,220.0
3,3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0
4,4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0


In [ ]:
df.shape

(1023, 11)

In [ ]:
# Экспресс-проверка на возможные пропуски. Количество строк после сброса пустых ячееек
# совпадает с количеством строк первоначального датафрейма - 1023.
df.dropna(how='all').count()

Unnamed: 0                              1023
Соотношение матрица-наполнитель         1023
Плотность, кг/м3                        1023
модуль упругости, ГПа                   1023
Количество отвердителя, м.%             1023
Содержание эпоксидных групп,%_2         1023
Температура вспышки, С_2                1023
Поверхностная плотность, г/м2           1023
Модуль упругости при растяжении, ГПа    1023
Прочность при растяжении, МПа           1023
Потребление смолы, г/м2                 1023
dtype: int64

In [ ]:
# Также, убедимся, что типы данных значений всех ячеек цифровые.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            1023 non-null   int64  
 1   Соотношение матрица-наполнитель       1023 non-null   float64
 2   Плотность, кг/м3                      1023 non-null   float64
 3   модуль упругости, ГПа                 1023 non-null   float64
 4   Количество отвердителя, м.%           1023 non-null   float64
 5   Содержание эпоксидных групп,%_2       1023 non-null   float64
 6   Температура вспышки, С_2              1023 non-null   float64
 7   Поверхностная плотность, г/м2         1023 non-null   float64
 8   Модуль упругости при растяжении, ГПа  1023 non-null   float64
 9   Прочность при растяжении, МПа         1023 non-null   float64
 10  Потребление смолы, г/м2               1023 non-null   float64
dtypes: float64(10), i

In [ ]:
# Создается дата-фрейм входных данных
df_input = df.iloc[:, [1, 2, 4, 5, -1]]
df_input.head(3)

,Соотношение матрица-наполнитель,"Плотность, кг/м3","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Потребление смолы, г/м2"
0,1.857143,2030.0,30.0,22.267857,220.0
1,1.857143,2030.0,50.0,23.750000,220.0
2,1.857143,2030.0,49.9,33.000000,220.0


In [ ]:
#  Создается дата-фрейм целевых признаков
df_target = df.iloc[:, [3, 6, 7, 8, 9]]
df_target.head(3)

,"модуль упругости, ГПа","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа"
0,738.736842,100.000000,210.0,70.0,3000.0
1,738.736842,284.615385,210.0,70.0,3000.0
2,738.736842,284.615385,210.0,70.0,3000.0


# 2. Разбивка данных на тренировочную и валидационную выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_input, df_target, train_size=0.8, random_state=42)

In [ ]:
# В дальнейшем планирую проработать разбивку методами Pytorch:

# train_idx_input, val_idx_input = train_test_split(list(range(len(df_input))), test_size=0.2, random_state=0)
# datasets_input = {}
# datasets_input['train'] = Subset(df_input, train_idx_input)
# datasets_input['val'] = Subset(df_input, val_idx_input)


# train_idx_target, val_idx_target = train_test_split(list(range(len(df_target))), test_size=0.2)
# datasets_target = {}
# datasets_target['train'] = Subset(df_target, train_idx_target)
# datasets_target['val'] = Subset(df_target, val_idx_target)

# 3. Нормирование данных и перевод их в тензоры


In [ ]:
# Создаем скейлеры
scaler_input = StandardScaler()
scaler_target = StandardScaler()

In [ ]:
# Далее создаем скалированные сеты тренировочных и тестовых данных
X_train_scaled = scaler_input.fit_transform(X_train.values)
y_train_scaled = scaler_target.fit_transform(y_train.values)
X_test_scaled = scaler_input.fit_transform(X_test.values)
y_test_scaled = scaler_target.fit_transform(y_test.values)

In [ ]:
# Перевод скалированных сетов в тензоры
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)


In [ ]:
X_train_tensor.shape

torch.Size([818, 5])

In [ ]:
y_train_tensor.shape

torch.Size([818, 5])

In [ ]:
# Задаются гиперпараметры модели
input_dim = X_train_tensor.shape[1]  # Размер входного тензора
output_dim = y_train_tensor.shape[1]  # Размер выходного тензора
d_model = input_dim  # Размерность модели должна быть равна размерности входных данных
nhead = 1  # Так как d_model = 7, выбираем nhead = 1, чтобы d_model % nhead == 0
num_encoder_layers = 3
dim_feedforward = 128
dropout = 0.1

In [ ]:
# Определение модели трансформера
class TransformerModel(nn.Module):
    def __init__(self, d_model, output_dim, nhead, num_encoder_layers, dim_feedforward, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
        self.fc_out = nn.Linear(d_model, output_dim)

    def forward(self, src):
        src = src.unsqueeze(1)  # Добавление batch dimension
        transformer_output = self.transformer_encoder(src)
        output = self.fc_out(transformer_output.squeeze(1))  # Удаление batch dimension
        return output


In [ ]:
# Инициализация модели
model = TransformerModel(d_model, output_dim, nhead, num_encoder_layers, dim_feedforward, dropout)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/home/mike-pi/miniconda3/envs/torchenv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Обучение модели
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Прямой проход
    outputs = model(X_train_tensor)

    # Вычисление потерь
    loss = criterion(outputs, y_train_tensor)

    # Обратный проход и оптимизация
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/1000], Loss: 1.0002
Epoch [20/1000], Loss: 1.0011
Epoch [30/1000], Loss: 0.9997
Epoch [40/1000], Loss: 0.9992
Epoch [50/1000], Loss: 0.9980
Epoch [60/1000], Loss: 0.9964
Epoch [70/1000], Loss: 0.9980
Epoch [80/1000], Loss: 0.9970
Epoch [90/1000], Loss: 0.9966
Epoch [100/1000], Loss: 0.9988
Epoch [110/1000], Loss: 0.9999
Epoch [120/1000], Loss: 0.9967
Epoch [130/1000], Loss: 1.0000
Epoch [140/1000], Loss: 0.9984
Epoch [150/1000], Loss: 0.9979
Epoch [160/1000], Loss: 0.9949
Epoch [170/1000], Loss: 0.9980
Epoch [180/1000], Loss: 0.9949
Epoch [190/1000], Loss: 0.9975
Epoch [200/1000], Loss: 0.9978
Epoch [210/1000], Loss: 0.9958
Epoch [220/1000], Loss: 0.9942
Epoch [230/1000], Loss: 0.9956
Epoch [240/1000], Loss: 0.9954
Epoch [250/1000], Loss: 0.9931
Epoch [260/1000], Loss: 0.9933
Epoch [270/1000], Loss: 0.9944
Epoch [280/1000], Loss: 0.9951
Epoch [290/1000], Loss: 0.9930
Epoch [300/1000], Loss: 0.9928
Epoch [310/1000], Loss: 0.9903
Epoch [320/1000], Loss: 0.9897
Epoch [330/1000],

In [ ]:
# Проверка модели на тестовых данных
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 1.0160
